# Tests for Diag 

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import yt
##yt.mylog.setLevel()
yt.utilities.logger.disable_stream_logging()

In [2]:
dataDir = '../single_level/'
baseName = 'plt'

In [3]:
ds = yt.load(dataDir + baseName + '0000001') # LOAD AT PLT 1 - intended that that corresponds to step==1

In [4]:
#ds.field_list

In [5]:
dx, dy, dz = ds.domain_width / ds.domain_dimensions # think this is ok for single level / UG

## diag_vel.out

Look at the file (assuming its only a few rows)

In [6]:
cat ../diag_vel.out

time                     max{U}                   max{Mach}                tot kin energy           tot grav energy          tot int energy           dt                       
0.000000000000000e+00    0.000000000000000e+00    0.000000000000000e+00    0.000000000000000e+00    2.642512900191456e+21    9.481602034658032e+30    1.974270210459415e-01    
1.974270210459415e-01    6.942426120264281e+01    9.746648957406819e-06    2.340851675917258e+20    2.642512900056286e+21    9.481602034521549e+30    1.974270210459415e-01    


Load up data needed for velocities and kinetic energy calculation

In [7]:
all_data_level_0 = ds.covering_grid(level=0, left_edge=[0,0.0,0.0], dims=[64, 64, 1])
vx = all_data_level_0['velx']
vx = vx[:,:,0].transpose()
vy = all_data_level_0['vely']
vy = vy[:,:,0].transpose()
rho = all_data_level_0['rho']
rho = rho[:,:,0].transpose()
w0y = all_data_level_0['w0y']
w0y = w0y[:,:,0].transpose()
vel = np.sqrt(vx**2 + (vy+w0y)**2)

Whats the maximum velocity according to the plotfile data?

In [8]:
np.max(vel) 

69.42426120264281 dimensionless

Whats the total kinetic energy according to the data in the plotfile? The codes diag file currently ignores the 0.5, and will "Normalise"?!? for dL, dA, or dV

In [9]:
ke = 0.5 * rho * vel**2
np.sum(2. * ke *dx * dy)

2.3408516759172606e+20 code_length**2

Whats the total internal energy according to the data in the plotfile? I had to look inside the EOS code to see how this is calculated

In [10]:
T = all_data_level_0['tfromp']
T = T[:,:,0].transpose()

In [11]:
k_B = 1.3806488e-16   # erg/K
n_A = 6.02214129e23  # mol^-1
R = k_B * n_A

mu = 1

gamma = 5./3.
cv = R /mu / (gamma-1)
eint = cv * T 

In [12]:
test = np.sum(eint*rho * dx**2)/9.481602034658032e30

In [13]:
np.float(test) -1 


-2.1264656702157936e-11

Whats the maximum mach number?

In [14]:
rho = all_data_level_0['rho']
rho = rho[:,:,0].transpose()
p = all_data_level_0['p0']
p = p[:,:,0].transpose()
cs = np.sqrt(gamma * p / rho)
mach = vel / cs

print( np.max(mach))

9.746648957452972e-06 dimensionless


### Whats the gravitational potential energy?

#### Definition

Note $g=-\frac{d \Phi(r)}{dr}$ with boundary condition $\Phi(r)=0$ (wlog) and constant gravity gives $\Phi=-gr$.

Note that $\Phi$ has units of length squared / time squared (in cgs cm^2/s^2) which can be thought of as energy per unit mass

So an energy density can be defined 

$\rho \Phi = - \rho g r$ 

This then can be evaluated with an appropriate volume integral (or area integral for a "2D energy")

#### Anayltical expectation for the lgw base state

The integral along the base state line is

$GE = -g \int_0^H \rho_0 r dr = -g \rho_{base} \int_0^H r e^{-r/H} dr $

$= -g \rho_{base} \left[-He^{-r/H}(H+R)\right]_0^{H}$

$= -g \rho_{base} H^2 [1-2e^{-1}] = -g \rho_{base} H^2 \times 0.26424111765$

This should then probabally be multiplied by the length in x to account for the rest of the integral over our 2d plane



For our implementation in maestro it seems pretty close, and I expect the difference is more to due with the numeric representation of our nominal base state rather than the way in which we calculate grav energy:

In [15]:
g=-3e4
H=1e9
r=1/3

U = -g * r * H**2 * (1-2*np.exp(-1))
print(U, U*1e9)

2.6424111765711534e+21 2.642411176571153e+30


In [16]:
Uout =2.642512900191456e21 #version manually entered copy and pasted from diag file
print(Uout / U)
print((Uout-U)/U)
print((Uout-U)*1e9)

1.0000384965145488
3.8496514548750144e-05
1.017236203026514e+26


## diag_temp.out

In [17]:
cat ../diag_temp.out

time                     max{T}                   x(max{T})                y(max{T})                z(max{T})                vx(max{T})               vy(max{T})               vz(max{T})               
0.000000000000000e+00    3.661246553124602e+05    7.812500000000000e+06    7.578125000000000e+08    0.000000000000000e+00    0.000000000000000e+00    0.000000000000000e+00    0.000000000000000e+00    
1.974270210459415e-01    3.661246543080655e+05    5.078125000000000e+08    7.578125000000000e+08    0.000000000000000e+00    2.208963925956074e-01    6.942390977405776e+01    0.000000000000000e+00    


In [18]:
T = all_data_level_0['tfromh']
T = T[:,:,0].transpose()
print(np.max(T))

366124.65430806554 dimensionless


can have a look at the actual coordinates, but its tricky if there are multiple maxima - might want to setup a test where one cell is very hot?

In [19]:
iy,ix = np.where( T == np.max(T))

In [20]:
T[iy, ix]

YTArray([366124.65430807]) (dimensionless)

In [21]:
ix * dx + dx/2

[5.078125e+08] code_length

In [22]:
iy * dy + dy/2

[7.578125e+08] code_length

In [23]:
vx[iy,ix]

YTArray([0.22089639]) (dimensionless)

In [24]:
vy[iy, ix] + w0y[iy,ix]

[69.42390977] dimensionless

## diag_enuc.out

Dont really have a good test for this. But since its so similar logically to temp, i think im willing to take it as OK for now (even to the point of making a PR). If there is a bug - it will get noticed later